In [ ]:
import pandas as pd
from src.api import get_league_data
from src.processing import extract_teams, extract_week_scores
from src.analysis import (
    add_ranks, filter_recent, build_owner_lookup,
    pivot_scores, normalize_per_column
)
from src.viz import (
    plot_heatmap, plot_line_chart, plot_boxplot
)

from src.config import config

LEAGUE_ID = config.league_id
YEARS = config.years
cookies = config.cookies
headers = config.headers
MY_TEAM_ID = 1

In [ ]:
all_scores = []
last_data = None

for year in YEARS:
    data = get_league_data(LEAGUE_ID, year, cookies=cookies, headers=headers)
    if not data:
        print(f"⚠️ Missing data for {year}, skipping.")
        continue

    last_data = data
    teams_df = extract_teams(data)
    scores = extract_week_scores(data, year)
    scores = scores.merge(teams_df, left_on="teamId", right_on="id").drop("id", axis=1)
    all_scores.append(scores)

In [ ]:
# Combine
df = pd.concat(all_scores, ignore_index=True)
df = add_ranks(df)
df = filter_recent(df, min_year=2020)

# Build owner lookup & pivot
owner_lookup = build_owner_lookup(last_data)
pivot = pivot_scores(df, owner_lookup)
normed = normalize_per_column(pivot)

In [ ]:
# === Graphs ===
plot_heatmap(normed, pivot, "week1-charts/plot1_heatmap_per_column.png")
plot_line_chart(df, splotlight_team=MY_TEAM_ID, outpath="week1-charts/plot2_line.png")
plot_boxplot(df, splotlight_team=MY_TEAM_ID, outpath="week1-charts/plot4_box.png")

In [ ]:
# Print my scores (debug)
df[df["teamId"] == MY_TEAM_ID]